In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np

from datetime import datetime, date
from configparser import ConfigParser

In [2]:
def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [3]:
conn = None
cur = None

# read connection parameters
params = config()

    # connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = pg.connect(**params)

    # create a cursor
cur = conn.cursor()

    # execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')

    # display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 16.3 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-12), 64-bit',)


In [4]:
# Now reading the tables from the database
cur.execute("SELECT * FROM sales_invoices")
rows = cur.fetchall()

type(rows)

list

In [5]:
sales_invoice = pd.DataFrame(rows, columns = ["id","store_id","billing_user_id","invoice_number","customer_id","customer_address_id","doctor_id","patient_id","order_type","is_home_delivery","total_amount_before_discount","discount_type","discount_percentage","discount_amount","igst","cgst","sgst","total_gst","round_off","total_invoice_amount","payment_status","total_amount_received","total_products","total_items","total_quantity","notes","is_active","created_at","created_by","updated_at","updated_by","deleted_at","deleted_by","sales_invoice_draft_id","prepaid_amount","device_master_id","taxable_amount","total_bill_amount","total_amount","voucher_discount_amount","total_discount","loyalty_points","loyalty_program_discount","is_loyalty_updated","is_customer_updated","promo_code","total_bill_amount_before_promo","is_sync_bill","is_prescription_required","delivery_charges","is_prescription_uploaded","prescription_uploaded_by","token","gst_treatment","gst_number","cin_number","place_of_supply_code","edited_user_id","whatsapp_sent_count"])
print(sales_invoice.shape)
sales_invoice.head()

(1419, 59)


,id,store_id,billing_user_id,invoice_number,customer_id,customer_address_id,doctor_id,patient_id,order_type,is_home_delivery,...,delivery_charges,is_prescription_uploaded,prescription_uploaded_by,token,gst_treatment,gst_number,cin_number,place_of_supply_code,edited_user_id,whatsapp_sent_count
0,571,36,120,SC2502000130,257,54.0,51.0,177.0,SALES,False,...,0.0,False,NaN,None,None,None,None,None,NaN,0
1,736,36,120,SC2502000218,257,54.0,51.0,177.0,SALES,False,...,0.0,False,NaN,None,None,None,None,None,NaN,1
2,621,36,120,SC2502000153,257,54.0,51.0,177.0,SALES,False,...,0.0,False,NaN,None,None,None,None,None,NaN,0
3,873,36,120,SC2502000306,144,48.0,51.0,172.0,SALES,False,...,0.0,False,NaN,None,None,None,None,None,NaN,0
4,657,36,120,SC2502000177,257,54.0,51.0,177.0,SALES,False,...,0.0,False,NaN,None,None,None,None,None,NaN,0


In [6]:
# Now reading the tables from the database
cur.execute("SELECT * FROM sales_invoice_details")
rows = cur.fetchall()

sales_invoice_details = pd.DataFrame(rows, columns = ["id","sales_invoice_id","product_id","batch_code","expiry_date","mrp","sales_rate","quantity","amount_before_discount","discount_type","discount_percentage","discount_amount","gst_percentage","gst_amount","total_amount","is_active","created_at","created_by","updated_at","updated_by","deleted_at","deleted_by","store_id","batch_id","base_rate","voucher_discount_amount","store_batch_id","is_advance_order","ordered_quantity","purchase_rate","device_master_id","bill_amount","loyalty_points","loyalty_program_discount","sales_rate_before_promo","is_prescription_required","scheduled_type_code","hsn_code"])

print(sales_invoice_details.shape)
sales_invoice_details.head()

(2792, 38)


,id,sales_invoice_id,product_id,batch_code,expiry_date,mrp,sales_rate,quantity,amount_before_discount,discount_type,...,ordered_quantity,purchase_rate,device_master_id,bill_amount,loyalty_points,loyalty_program_discount,sales_rate_before_promo,is_prescription_required,scheduled_type_code,hsn_code
0,1327,952,21469,DOTL922,2025-01-31,44.35,42.13,1,0.0,%,...,0,0.0,30.0,42.13,0.89,0.0,0.0,False,None,None
1,1351,968,29341,BELT1,2028-12-31,570.00,552.90,2,0.0,%,...,0,0.0,61.0,1105.80,0.0,0.0,0.0,False,None,None
2,1352,968,11734,22050261,2027-04-30,3.00,2.82,2,0.0,%,...,0,0.0,61.0,5.64,0.0,0.0,0.0,False,None,None
3,683,565,14166,MPC230703,2025-02-28,40.50,34.43,1,0.0,%,...,0,0.0,796.0,NaN,0.0,NaN,0.0,False,None,None
4,684,565,24054,AMP23012,2028-01-31,105.00,89.25,1,0.0,%,...,0,0.0,796.0,NaN,0.0,NaN,0.0,False,None,None


In [7]:
cur.execute("SELECT * FROM sales_return")
rows = cur.fetchall()

sales_return = pd.DataFrame(rows, columns=["id","sales_invoice_id","customer_id","store_id","credit_note_number","total_quantity","total_products","billing_user_id","total_amount","bill_amount","total_gst","taxable_amount","cgst","sgst","igst","refund_amount","round_off","created_at","created_by","updated_at","updated_by","deleted_at","deleted_by","invoice_number","total_items","device_master_id","loyalty_points","return_type","remarks","delivery_charges","is_sync_return","gst_treatment","gst_number","cin_number","place_of_supply_code"])

print(sales_return.shape)
sales_return.head()

(305, 35)


,id,sales_invoice_id,customer_id,store_id,credit_note_number,total_quantity,total_products,billing_user_id,total_amount,bill_amount,...,device_master_id,loyalty_points,return_type,remarks,delivery_charges,is_sync_return,gst_treatment,gst_number,cin_number,place_of_supply_code
0,207,1422.0,272,36,SR2502000090,5,4,79,1415.00,1297.35,...,830.0,4,SALES_RETURN,None,0,False,None,None,None,None
1,2,152.0,1,28,CN123654,10,2,72,100.00,80.00,...,NaN,0.0,SALES_RETURN,None,0,False,None,None,None,None
2,213,1365.0,136,36,SR2502000095,1,1,40,106.48,90.51,...,586.0,0,SALES_RETURN,None,None,False,None,None,None,None
3,219,NaN,79,29,SR2501800042,15,1,82,1500.00,1405.00,...,44.0,0,OPEN_RETURN,testing open return,50,False,None,None,None,None
4,275,NaN,276,36,SR2502000133,21,2,67,501.60,381.60,...,208.0,0,OPEN_RETURN,dsgsdgsd,20,False,None,None,None,None


In [8]:
cur.execute("SELECT * FROM products")
rows = cur.fetchall()

products = pd.DataFrame(rows, columns = ["id","product_name","product_type","ws_code","product_code","mrp","is_assured","is_discontinued","is_banned","is_rx_required","is_chronic","is_refrigerated","scheduled_type_code","mis_reporting_category","is_active","dosage_form","package_type","uom","package_size","sales_unit","gst_type","hsn_code","b_2_b_product_type","sales_trend_category","vendor_return_type","purchase","purchase_return","transfer_in","transfer_out","franchise_in","franchise_out","b2c_in","b2c_out","created_by","created_at","updated_by","updated_at","deleted_by","deleted_at","wms_product_id","manufacturer_name","combinations_string","marketed_by","sales_price","b_2_c_product_category_id","b_2_c_product_type","is_alternate_available","alternate_product_id","combinations_string_slug","is_hidden_from_alternate_products"])

print(products.shape)
products.head()

(18870, 50)


,id,product_name,product_type,ws_code,product_code,mrp,is_assured,is_discontinued,is_banned,is_rx_required,...,manufacturer_name,combinations_string,marketed_by,sales_price,b_2_c_product_category_id,b_2_c_product_type,is_alternate_available,alternate_product_id,combinations_string_slug,is_hidden_from_alternate_products
0,11376,A MOR Z TAB 1X15,Goods,17.0,37984,100,True,False,False,False,...,INCY HEALTHCARE PVT LTD,ELEMENTAL COPPER 0.9 MG+ELEMENTAL MANGANESE 2 ...,None,None,6,GENERIC,True,27804.0,elemental-copper-09-mgelemental-manganese-2-mg...,False
1,11390,A RET 0.025% GEL 20GM,Goods,18.0,37998,101,False,True,False,True,...,A. MENARINI INDIA PRIVATE LIMITED,TRETINOIN 0.025 %W/W,None,None,5,Service,True,20072.0,tretinoin-0025-ww,False
2,11399,A RET HC CREAM 15GM,Goods,20.0,38007,166,False,True,False,True,...,A. MENARINI INDIA PRIVATE LIMITED,HYDROCORTISONE ACETATE 1 %W/W+HYDROQUINONE 2 %...,None,None,5,Service,True,25667.0,hydrocortisone-acetate-1-wwhydroquinone-2-wwtr...,False
3,11400,A TO Z TAB 1X15,Goods,25.0,38008,128,False,False,False,True,...,ALKEM LABORATORIES LIMITED,ELEMENTAL COPPER 0.9 MG+ELEMENTAL MANGANESE 2 ...,None,None,5,Service,True,27804.0,elemental-copper-09-mgelemental-manganese-2-mg...,False
4,11419,AB FLO SR 200MG TAB 1X10,Goods,40.0,38027,245.6,False,False,False,True,...,LUPIN LIMITED,ACEBROPHYLLINE 200 MG SUSTAINED RELEASE,None,None,5,Service,True,18330.0,acebrophylline-200-mg-sustained-release,False


In [9]:
cur.execute("SELECT * FROM users")
rows = cur.fetchall()

users = pd.DataFrame(rows, columns=["id","designation_id","name","mobile_number","password","image","is_active","created_by","updated_by","deleted_by","created_at","updated_at","deleted_at","refresh_token","email","salutation","mre_fcm_token","pos_fcm_token","employee_code","mre_user_id"])
print(users.shape)
users.head()

(329, 20)


,id,designation_id,name,mobile_number,password,image,is_active,created_by,updated_by,deleted_by,created_at,updated_at,deleted_at,refresh_token,email,salutation,mre_fcm_token,pos_fcm_token,employee_code,mre_user_id
0,753,2.0,Krishnapharmacy,7624015943,None,None,True,82.0,82.0,NaN,NaT,2024-09-12 13:07:27,NaT,None,admin2@gmail.com,None,None,None,None,4.0
1,754,2.0,NILKANTHPHARMACY2,7417007112,None,None,True,82.0,82.0,NaN,NaT,2024-09-12 13:07:27,NaT,None,nilkanthatladara@gmail.com,None,None,None,None,25.0
2,832,326.0,test accountant,7788996633,$2y$12$IpI0WQiD8o8AA5OM2uQnIu55yNEHpKgRbwtBS3/...,None,True,78.0,NaN,NaN,2024-09-25 16:17:50,2024-10-15 11:55:11,NaT,None,accountant.a@medkart.in,Mr.,c4EGLLVtQGu9srEbtThf-C:APA91bEbtZEPPLWaJFlOVQu...,None,101020,387.0
3,835,11.0,VIDHI ADMIN -- MEDKART B AKOTA,6352416352,$2y$12$AODUbDpHZjEHUR0IDI/dvu5Phyj.nEZ0Iuh5hXv...,None,True,31.0,NaN,NaN,2024-10-22 12:37:58,2024-10-22 12:37:59,NaT,None,None,Mrs.,None,None,15969,395.0
4,78,11.0,SHREYANSH - ADMIN,7984481792,$2y$12$9omAzhwcO7UyNcW1W2mv5eRMvZer2rTRbhg.V0a...,None,True,31.0,82.0,NaN,2024-05-15 12:19:24,2024-11-07 17:05:12,NaT,None,ventureusertest@medkart.in,Mr.,None,fzeleW8rQsu3SVuDmHhbZD:APA91bGqJE1fY6U5F7vuTUA...,12345678,234.0


In [10]:
cur.execute("SELECT * FROM customers")
rows = cur.fetchall()

customers = pd.DataFrame(rows, columns=["id","full_name","mobile_number","alternate_mobile_number","email_address","customer_since","is_generic_buyer","referred_by","referral_code","is_loyalty_enrolled","is_active","created_at","created_by","updated_at","updated_by","deleted_at","deleted_by","is_restricted","ltv","aov","no_of_bills","store_type","purchase_channel","loyalty_points","loyalty_register_invoice_id","loyalty_register_invoice_amount","loyalty_expires_at","is_branded_buyer","customer_category","generic_ltv","branded_ltv","otc_ltv","chronic_ltv","acute_ltv","first_purchase_amount","last_purchase_amount","first_purchase_store_id","last_purchase_store_id","last_purchase_bill_date","is_message_sent","customer_code","loyalty_enrolled_at","loyalty_register_store_id"])
print(customers.shape)
customers.head()

(226, 43)


,id,full_name,mobile_number,alternate_mobile_number,email_address,customer_since,is_generic_buyer,referred_by,referral_code,is_loyalty_enrolled,...,acute_ltv,first_purchase_amount,last_purchase_amount,first_purchase_store_id,last_purchase_store_id,last_purchase_bill_date,is_message_sent,customer_code,loyalty_enrolled_at,loyalty_register_store_id
0,128,Charmi Kalayani,6352733661,None,None,2024-02-12,False,NaN,None,False,...,0,0,0,NaN,NaN,None,False,None,None,NaN
1,367,sdcszcfsa,7896541330,None,None,2024-12-03,False,NaN,None,False,...,0,0,0,NaN,NaN,None,False,CS0000000017,None,NaN
2,361,FDG,9878987879,None,None,2024-09-18,False,NaN,None,False,...,0,0,0,NaN,NaN,None,False,CS0000000011,None,NaN
3,279,Hulk,6354028567,None,None,2024-06-22,False,NaN,None,False,...,0,0,0,NaN,NaN,None,False,None,None,NaN
4,282,Thor,6354028561,None,None,2024-06-22,False,NaN,None,False,...,0,0,0,NaN,NaN,None,False,None,None,NaN


In [11]:
cur.execute("SELECT * FROM stores")
rows = cur.fetchall()

stores = pd.DataFrame(rows, columns=["id","name","store_type","wms_store_id","ws_store_id","ws_alternate_code","is_active","created_by","updated_by","deleted_by","created_at","updated_at","deleted_at","gmb_place_id","gst_number","store_group","address_1","address_2","address_3","pincode","city","state","latitude","longitude","place_of_supply_code","is_custom_location","is_loyalty_enrolled","is_pos_applicable","gst_treatment","cin_number","gmb_store_name","is_whatsapp_bill_applicable"])
print(stores.shape)
stores.head()

(124, 32)


,id,name,store_type,wms_store_id,ws_store_id,ws_alternate_code,is_active,created_by,updated_by,deleted_by,...,latitude,longitude,place_of_supply_code,is_custom_location,is_loyalty_enrolled,is_pos_applicable,gst_treatment,cin_number,gmb_store_name,is_whatsapp_bill_applicable
0,73,MEDKART A IOC ROAD,COCO,250,125,125,True,6,6,None,...,23.012616,72.510939,24,False,False,True,Regular,None,MEDKART PHARMACY,True
1,160,test store updated 1,FOFO,376,198,963,True,547,547,None,...,23.0225,72.5714,24,False,False,False,Regular,24AARFC9888M1ZC,None,True
2,164,test store 242,FOFO,388,215,963,True,82,82,None,...,23.0225,72.5714,24,False,False,False,Regular,asdfasfasdf,None,True
3,74,MEDKART B CHHANI JAKATNAKA,COCO,252,23,23,True,6,6,None,...,22.3479119,73.174264,24,False,False,True,Regular,None,MEDKART PHARMACY,True
4,52,MAHALAXMI ENTERPRISE NIKOL,FOFO,286,90,90,True,6,6,None,...,23.012616,72.510939,24,True,False,True,Composition,None,MEDKART PHARMACY,True


In [12]:
cur.execute("SELECT * FROM store_inventories")
rows = cur.fetchall()

store_inventories = pd.DataFrame(rows, columns=["id","store_inventory_transaction_id","store_id","product_id","batch_code","opening_stock","quantity","closing_stock","is_active","created_at","created_by","updated_at","updated_by","deleted_at","deleted_by","store_batch_id"])

print(store_inventories.shape)
store_inventories.head()

(46177, 16)


,id,store_inventory_transaction_id,store_id,product_id,batch_code,opening_stock,quantity,closing_stock,is_active,created_at,created_by,updated_at,updated_by,deleted_at,deleted_by,store_batch_id
0,38503,38524,36,12041,KAE2207A,0,569,569,False,2024-06-06 10:24:46.910545,75,2024-06-06 10:24:46.910545,None,2024-06-18 09:37:13.186430,97.0,14749
1,24402,24422,28,14913,M659,0,928,928,True,2024-06-06 10:22:44.397615,75,2024-06-06 10:22:44.397615,None,NaT,NaN,1935
2,76729,76785,23,21474,DOBS3174,0,3,3,False,2024-09-10 08:59:44.125490,63,2024-09-10 08:59:44.125490,None,2024-09-11 07:35:55.760087,62.0,42953
3,24403,24423,28,23371,4S43,0,888,888,True,2024-06-06 10:22:44.397615,75,2024-06-06 10:22:44.397615,None,NaT,NaN,2088
4,56822,56870,130,18593,SN21018,0,46,46,False,2024-06-17 09:21:08.614448,75,2024-06-17 09:21:08.614448,None,2024-09-13 14:00:16.056925,65.0,34445


In [13]:
cur.execute("SELECT * FROM store_inventory_transactions")
rows = cur.fetchall()

store_inventory_transactions = pd.DataFrame(rows, columns=["id","store_id","product_id","batch_code","expiry_date","transaction_time","quantity","rate","total_amount","voucher_type","voucher_id","is_reverse","reverse_id","is_active","created_at","created_by","updated_at","updated_by","deleted_at","deleted_by","store_batch_id"])

print(store_inventory_transactions.shape)
store_inventories.head()

(66613, 21)


,id,store_inventory_transaction_id,store_id,product_id,batch_code,opening_stock,quantity,closing_stock,is_active,created_at,created_by,updated_at,updated_by,deleted_at,deleted_by,store_batch_id
0,38503,38524,36,12041,KAE2207A,0,569,569,False,2024-06-06 10:24:46.910545,75,2024-06-06 10:24:46.910545,None,2024-06-18 09:37:13.186430,97.0,14749
1,24402,24422,28,14913,M659,0,928,928,True,2024-06-06 10:22:44.397615,75,2024-06-06 10:22:44.397615,None,NaT,NaN,1935
2,76729,76785,23,21474,DOBS3174,0,3,3,False,2024-09-10 08:59:44.125490,63,2024-09-10 08:59:44.125490,None,2024-09-11 07:35:55.760087,62.0,42953
3,24403,24423,28,23371,4S43,0,888,888,True,2024-06-06 10:22:44.397615,75,2024-06-06 10:22:44.397615,None,NaT,NaN,2088
4,56822,56870,130,18593,SN21018,0,46,46,False,2024-06-17 09:21:08.614448,75,2024-06-17 09:21:08.614448,None,2024-09-13 14:00:16.056925,65.0,34445


----------------------------

## MAIN LOGIC

In [14]:
date_today = date.today()
print(date_today)

date_today = date(2024, 10, 31)
date_today

2025-02-03


datetime.date(2024, 10, 31)

In [15]:
# Get the first date of the month for the current date
first_date_of_month = date_today.replace(day=1)
print(first_date_of_month)

2024-10-01


In [16]:
# Convert dates to datetime 
sales_invoice['created_at'] = pd.to_datetime(sales_invoice['created_at'])
first_date_of_month = pd.to_datetime(first_date_of_month)
date_today = pd.to_datetime(date_today)

# Filter with proper syntax - use parentheses and & instead of and
filtered_sales = sales_invoice[
    (sales_invoice['created_at'] >= first_date_of_month) & 
    (sales_invoice['created_at'] <= date_today)
]

In [17]:
# Convert dates to datetime 
sales_return['created_at'] = pd.to_datetime(sales_return['created_at'])
first_date_of_month = pd.to_datetime(first_date_of_month)
date_today = pd.to_datetime(date_today)

# Filter with proper syntax - use parentheses and & instead of and
filtered_sales_r = sales_return[
    (sales_return['created_at'] >= first_date_of_month) & 
    (sales_return['created_at'] <= date_today)
]

In [18]:
filtered_sales.head()

,id,store_id,billing_user_id,invoice_number,customer_id,customer_address_id,doctor_id,patient_id,order_type,is_home_delivery,...,delivery_charges,is_prescription_uploaded,prescription_uploaded_by,token,gst_treatment,gst_number,cin_number,place_of_supply_code,edited_user_id,whatsapp_sent_count
77,1786,36,40,SC2502000918,272,153.0,51.0,353.0,SALES,False,...,0.0,False,NaN,None,None,None,None,None,NaN,0
694,1788,24,96,SC2500600018,262,62.0,61.0,198.0,SALES,False,...,0.0,False,NaN,None,None,None,None,None,NaN,0
855,1789,24,96,SC2500600019,143,67.0,49.0,202.0,SALES,True,...,10.0,False,NaN,None,None,None,None,None,NaN,0
996,1790,24,96,SC2500600020,143,67.0,49.0,202.0,SALES,True,...,10.0,False,NaN,None,None,None,None,None,NaN,0
1063,1791,36,60,SC2502000920,360,NaN,65.0,373.0,SALES,False,...,0.0,False,NaN,None,None,None,None,None,NaN,0


In [19]:
filtered_sales = filtered_sales[['id', 'store_id', 'invoice_number', 'total_amount', 'customer_id', 'total_products', 'total_items', 'created_at']]
filtered_sales.head()

,id,store_id,invoice_number,total_amount,customer_id,total_products,total_items,created_at
77,1786,36,SC2502000918,1440.00,272,1,1,2024-10-25 14:58:31
694,1788,24,SC2500600018,154.55,262,1,1,2024-10-25 15:56:51
855,1789,24,SC2500600019,350.01,143,2,2,2024-10-25 16:35:36
996,1790,24,SC2500600020,257.28,143,2,2,2024-10-25 16:45:11
1063,1791,36,SC2502000920,410.00,360,2,2,2024-10-25 17:12:15


In [20]:
filtered_sales_r = filtered_sales_r[['id', 'store_id', 'credit_note_number', 'total_amount', 'customer_id', 'total_products', 'total_items', 'created_at']]
filtered_sales_r = filtered_sales_r.rename(columns={'credit_note_number':'invoice_number'})
filtered_sales_r.head()

,id,store_id,invoice_number,total_amount,customer_id,total_products,total_items,created_at
27,328,36,SR2502000170,637.50,276,1,1.0,2024-10-03 11:23:11
28,332,36,SR2502000174,637.50,276,1,1.0,2024-10-03 11:27:36
29,338,36,SR2502000180,637.50,276,1,1.0,2024-10-07 12:11:21
30,349,36,SR2502000191,286.94,276,1,1.0,2024-10-14 17:42:19
37,352,36,SR2502000194,865.00,137,2,2.0,2024-10-18 16:47:01


In [21]:
# Append the two dataframes
filtered_sales = pd.concat([filtered_sales, filtered_sales_r])
filtered_sales.to_csv("filtered_2.csv")

In [22]:
# Now merge with the stores df and get the type of store here.
filtered_sales = pd.merge(filtered_sales, stores[['id', 'name', 'store_type']], left_on='store_id', right_on='id', how='left')
filtered_sales.head()

,id_x,store_id,invoice_number,total_amount,customer_id,total_products,total_items,created_at,id_y,name,store_type
0,1786,36,SC2502000918,1440.00,272,1,1.0,2024-10-25 14:58:31,36,MEDKART B AKOTA,COCO
1,1788,24,SC2500600018,154.55,262,1,1.0,2024-10-25 15:56:51,24,MEDKART A PRERNATIRTH,COCO
2,1789,24,SC2500600019,350.01,143,2,2.0,2024-10-25 16:35:36,24,MEDKART A PRERNATIRTH,COCO
3,1790,24,SC2500600020,257.28,143,2,2.0,2024-10-25 16:45:11,24,MEDKART A PRERNATIRTH,COCO
4,1791,36,SC2502000920,410.00,360,2,2.0,2024-10-25 17:12:15,36,MEDKART B AKOTA,COCO


In [23]:
# Now from the created_at column remove the time part and keep only the date part
filtered_sales['created_at'] = filtered_sales['created_at'].dt.date
filtered_sales.head()

,id_x,store_id,invoice_number,total_amount,customer_id,total_products,total_items,created_at,id_y,name,store_type
0,1786,36,SC2502000918,1440.00,272,1,1.0,2024-10-25,36,MEDKART B AKOTA,COCO
1,1788,24,SC2500600018,154.55,262,1,1.0,2024-10-25,24,MEDKART A PRERNATIRTH,COCO
2,1789,24,SC2500600019,350.01,143,2,2.0,2024-10-25,24,MEDKART A PRERNATIRTH,COCO
3,1790,24,SC2500600020,257.28,143,2,2.0,2024-10-25,24,MEDKART A PRERNATIRTH,COCO
4,1791,36,SC2502000920,410.00,360,2,2.0,2024-10-25,36,MEDKART B AKOTA,COCO


In [24]:
# Now filter all the store_types where type = "FOFO"
filtered_sales = filtered_sales[filtered_sales['store_type'] == "COCO"]
filtered_sales.head()

,id_x,store_id,invoice_number,total_amount,customer_id,total_products,total_items,created_at,id_y,name,store_type
0,1786,36,SC2502000918,1440.00,272,1,1.0,2024-10-25,36,MEDKART B AKOTA,COCO
1,1788,24,SC2500600018,154.55,262,1,1.0,2024-10-25,24,MEDKART A PRERNATIRTH,COCO
2,1789,24,SC2500600019,350.01,143,2,2.0,2024-10-25,24,MEDKART A PRERNATIRTH,COCO
3,1790,24,SC2500600020,257.28,143,2,2.0,2024-10-25,24,MEDKART A PRERNATIRTH,COCO
4,1791,36,SC2502000920,410.00,360,2,2.0,2024-10-25,36,MEDKART B AKOTA,COCO


In [25]:
def bill_type(invoice_num):
    # If the invoice number has SC then SC else SR
    if "SC" in invoice_num:
        return "SC"
    else:
        return "SR"
    
filtered_sales['bill_type'] = filtered_sales['invoice_number'].apply(bill_type)
filtered_sales.head()

,id_x,store_id,invoice_number,total_amount,customer_id,total_products,total_items,created_at,id_y,name,store_type,bill_type
0,1786,36,SC2502000918,1440.00,272,1,1.0,2024-10-25,36,MEDKART B AKOTA,COCO,SC
1,1788,24,SC2500600018,154.55,262,1,1.0,2024-10-25,24,MEDKART A PRERNATIRTH,COCO,SC
2,1789,24,SC2500600019,350.01,143,2,2.0,2024-10-25,24,MEDKART A PRERNATIRTH,COCO,SC
3,1790,24,SC2500600020,257.28,143,2,2.0,2024-10-25,24,MEDKART A PRERNATIRTH,COCO,SC
4,1791,36,SC2502000920,410.00,360,2,2.0,2024-10-25,36,MEDKART B AKOTA,COCO,SC


In [26]:
# Sort by store_id and then by created_date
filtered_sales = filtered_sales.sort_values(by=['created_at', 'store_id'])
filtered_sales.head()

,id_x,store_id,invoice_number,total_amount,customer_id,total_products,total_items,created_at,id_y,name,store_type,bill_type
7,1695,36,SC2502000848,255.0,276,1,1.0,2024-10-03,36,MEDKART B AKOTA,COCO,SC
116,328,36,SR2502000170,637.5,276,1,1.0,2024-10-03,36,MEDKART B AKOTA,COCO,SR
117,332,36,SR2502000174,637.5,276,1,1.0,2024-10-03,36,MEDKART B AKOTA,COCO,SR
121,329,36,SR2502000171,637.5,276,1,1.0,2024-10-03,36,MEDKART B AKOTA,COCO,SR
122,333,36,SR2502000175,937.5,276,2,2.0,2024-10-03,36,MEDKART B AKOTA,COCO,SR


In [27]:
filtered_sales = filtered_sales.groupby(['name', 'created_at', 'bill_type']).agg({'total_amount':'sum'}).reset_index()
# typecast the created_at column to a string type but with the same value
filtered_sales['created_at'] = filtered_sales['created_at'].astype(str)
filtered_sales.head()

,name,created_at,bill_type,total_amount
0,MEDKART A GURUKUL,2024-10-10,SC,1720.0
1,MEDKART A GURUKUL,2024-10-14,SC,1720.0
2,MEDKART A GURUKUL,2024-10-15,SC,14500.0
3,MEDKART A PARIMAL,2024-10-07,SC,1550.0
4,MEDKART A PARIMAL,2024-10-08,SC,310.0


In [28]:
filtered_sales = filtered_sales.groupby(['name', 'created_at', 'bill_type']).agg({'total_amount':'sum'}).unstack([1, 2]).sort_index(axis=1, level=1)

In [29]:
filtered_sales.to_excel("Final_output.xlsx")

In [30]:
# Store in xls in such a way that the created at row is in date format


## PART 2

In [31]:
part2 = sales_invoice[['id', 'store_id', 'invoice_number', 'total_amount', 'customer_id', 'total_products', 'total_items', 'created_at']]
sales_return = sales_return[['id', 'store_id', 'credit_note_number', 'total_amount', 'customer_id', 'total_products', 'total_items', 'created_at']]
sales_return = sales_return.rename(columns={'credit_note_number':'invoice_number'})

part2 = pd.concat([part2, sales_return])
part2.head()

,id,store_id,invoice_number,total_amount,customer_id,total_products,total_items,created_at
0,571,36,SC2502000130,1031.02,257,2,2.0,2024-06-20 16:59:41
1,736,36,SC2502000218,770.72,257,2,2.0,2024-06-25 11:23:01
2,621,36,SC2502000153,1086.00,257,2,2.0,2024-06-21 19:07:02
3,873,36,SC2502000306,88.70,144,1,1.0,2024-07-12 18:17:55
4,657,36,SC2502000177,1738.64,257,2,2.0,2024-06-24 12:22:09


In [32]:
# Extract the month and year from the created_at using the strftime function
part2['created_at'] = pd.to_datetime(part2['created_at'])
part2['month'] = part2['created_at'].dt.strftime('%Y-%m')

part2.head()

,id,store_id,invoice_number,total_amount,customer_id,total_products,total_items,created_at,month
0,571,36,SC2502000130,1031.02,257,2,2.0,2024-06-20 16:59:41,2024-06
1,736,36,SC2502000218,770.72,257,2,2.0,2024-06-25 11:23:01,2024-06
2,621,36,SC2502000153,1086.00,257,2,2.0,2024-06-21 19:07:02,2024-06
3,873,36,SC2502000306,88.70,144,1,1.0,2024-07-12 18:17:55,2024-07
4,657,36,SC2502000177,1738.64,257,2,2.0,2024-06-24 12:22:09,2024-06


In [33]:
date_today = date(2024,7,1)
date_today

datetime.date(2024, 7, 1)

In [34]:
date_today

datetime.date(2024, 7, 1)

In [35]:
# Now I want to filter the previous month of the date_today. Means if date today is 2024-10-31, then it should return 2024-09
prev_month = date_today.replace(day=1) - pd.DateOffset(months=1)
prev_month = prev_month.strftime('%Y-%m')
prev_month

'2024-06'

In [36]:
# Now drop the created_at column and then take all the part2 months equal to the previous months
part2 = part2.drop(columns=['created_at'])
part2 = part2[part2['month'] == prev_month]
part2

,id,store_id,invoice_number,total_amount,customer_id,total_products,total_items,month
0,571,36,SC2502000130,1031.02,257,2,2.0,2024-06
1,736,36,SC2502000218,770.72,257,2,2.0,2024-06
2,621,36,SC2502000153,1086.00,257,2,2.0,2024-06
4,657,36,SC2502000177,1738.64,257,2,2.0,2024-06
5,714,36,SC2502000203,770.72,257,2,2.0,2024-06
...,...,...,...,...,...,...,...,...
216,131,36,SR2502000046,12.00,137,1,1.0,2024-06
218,133,29,SR2501800032,168.00,1,1,1.0,2024-06
219,134,36,SR2502000047,493.00,257,1,1.0,2024-06
220,135,36,SR2502000048,770.72,257,2,2.0,2024-06


In [37]:
# Now merge the stores and then filter the store_type = "FOFO"
part2 = pd.merge(part2, stores[['id', 'name', 'store_type']], left_on='store_id', right_on='id', how='left')
part2 = part2[part2['store_type'] == "FOFO"]
part2

,id_x,store_id,invoice_number,total_amount,customer_id,total_products,total_items,month,id_y,name,store_type
7,593,141,SC2502000144,225.10,90,1,1.0,2024-06,141,NILKANTH PHARMA,FOFO
9,718,141,SC2502000207,163.60,1,1,1.0,2024-06,141,NILKANTH PHARMA,FOFO
10,641,141,SC2502000166,1744.06,1,1,1.0,2024-06,141,NILKANTH PHARMA,FOFO
11,698,141,SC2502000197,885.00,239,1,1.0,2024-06,141,NILKANTH PHARMA,FOFO
12,795,141,SC2502000242,30.07,1,1,1.0,2024-06,141,NILKANTH PHARMA,FOFO


In [38]:
# Group by store name and total the amount for every store
part2 = part2.groupby(['name']).agg({'total_amount':'sum'}).reset_index()
part2

,name,total_amount
0,NILKANTH PHARMA,3047.83


In [39]:
part2['RL'] = part2['total_amount'] * 0.02
part2

,name,total_amount,RL
0,NILKANTH PHARMA,3047.83,60.9566
